## Description: prepare the notebook for manual annotation

In this notebook, three elements are added to the previously generated dataset:
- check that the chapter and verse_num columns are integers
- sign_info column, based on a recent version of the function
- reconstructed_verse column which is a gcons version of the verse with brackets indicating the reconstructed sections
- the spatial_arg_type column for annotating goal, source, location, etc. 
- remove the rows which have "MN" in preposition_1 and no preposition_2

In [1]:
import pandas as pd

from itertools import chain

from tfob import TFOb,  get_dss

DSS = get_dss()

PATH = 'data/biblical_datasets/psalms/annotation_df_history/Psalms_dataset.csv' # Modify before executing the notebook

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [2]:
#df_1 = pd.read_csv('../2_datasets/generate_datasets/data/biblical_datasets/Jeremiah_dataset.csv').fillna("")
df = pd.read_csv(PATH).fillna("")

In [3]:
# Make the values in "chapter" and "verse_num" integers
df['chapter'] = df['chapter'].astype(int)
df['verse_num'] = df['verse_num'].astype(int)

In [4]:
# Reorganise the dataset by chapter, verse_num and scroll

# Custom sorting logic for "scroll" column
df['scroll_order'] = df['scroll'].apply(lambda x: 0 if x == "MT" else 1)

# Sort by "scroll_order", "chapter", and "verse_num"
df = df.sort_values(by=['chapter', 'verse_num', 'scroll_order'])

# Drop the temporary 'scroll_order' column
df = df.drop(columns='scroll_order')

df = df.reset_index(drop=True)

In [5]:
pd.set_option("display.max_columns", None)

In [6]:
df.head(20)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments
0,310658,HLK[,MT,Psalms,1,1,HLK,>CRJ H >JC >CR L> HLK B <YT RC<JM W B DRK XV>J...,>CR L> HLK B <YT RC<JM,,B <YT RC<JM,B <YH/ RC</,0.0,prep,,,det,subs,complex,,,qal,perf,B,,,,,,,,,
1,310854,XSH[,MT,Psalms,2,12,XWSJ,NCQW BR PN J>NP W T>BDW DRK KJ JB<R K M<V >PW ...,XWSJ BW,,BW,B,0.0,prep,,,det,,simple,,,qal,ptca,B,,,,,,,,,
2,310860,BRX[,MT,Psalms,3,1,BRXW,MZMWR L DWD B BRXW M PNJ >BCLWM BNW,B BRXW M PNJ >BCLWM BNW,,M PNJ >BCLWM BNW,MN PNH/ >BCLWM/ BN/,0.0,prep,pers,anim,det,subs,complex,,,qal,infc,MN,,,,,,,,,
3,311104,BW>[,MT,Psalms,5,8,>BW>,W >NJ B RB XSDK >BW> BJTK >CTXWH >L HJKL QDCK ...,W >NJ B RB XSDK >BW> BJTK,>NJ,BJTK,BJT/,0.0,vc,,,det,subs,simple,,,qal,impf,,,,,,,,,,
4,311136,NPL[,MT,Psalms,5,11,JPLW,H>CJMM >LHJM JPLW M M<YWTJHM B RB PC<JHM HDJXM...,JPLW M M<YWTJHM,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,
5,311149,XSH[,MT,Psalms,5,12,XWSJ,W JFMXW KL XWSJ BK L <WLM JRNNW W TSK <LJMW W ...,XWSJ BK,,BK,B,0.0,prep,,,det,,simple,,,qal,ptca,B,,,,,,,,,
6,2054155,XSH[,4Q98b,Psalms,5,12,XSJ,W JFMXW KL XSJ BKH L <WLM JRNNW W TSKH <LJMW W...,XSJ BKH,,BKH,,0.0,,,,,,,0.0,0 00000 00 000 000 0 0000 00000 0 0000 00000 0...,qal,ptca,BKH,,,,,,,,,
7,2046935,XSH[,4Q83,Psalms,5,12,XSJ,W JFMXW KWL XSJ BKH L <WLM JRNNW W TSK <LJMW W...,XSJ BKH,,BKH,,0.0,,,,,,,0.0,0 00000 000 000 000 0 0000 00000 0 000 00000 0...,qal,ptca,BKH,,,,,,,,,
8,311212,CWB[,MT,Psalms,6,5,CWBH,CWBH JHWH XLYH NPCJ HWCJ<NJ LM<N XSDK,CWBH,,no complement,no complement,,,,,,,,,,qal,impv,,,,,,,,,,
9,311250,SWR[,MT,Psalms,6,9,SWRW,SWRW MMNJ KL P<LJ >WN KJ CM< JHWH QWL BKJJ,SWRW MMNJ,,MMNJ,MN,0.0,prep,,,det,,simple,,,qal,impv,MN,,,,,,,,,


In [7]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments'],
      dtype='object')

In [8]:
# This is the most recent sign_info function which uses the .rec (1 October 2024)

def sign_info(verb):
    sign_rec_words = []
    
    scroll = verb.to_scrolls.scroll[0]
    verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
    
    for word in verse.to_words:
        signs = word.to_signs
        sign_rec = []
        for sign in signs:
            if sign.type[0] == "cons": # only keeps consonants
                if sign.rec[0]:
                    sign_rec.append("1")
                else:
                    sign_rec.append("0")
        if sign_rec:
            sign_rec_words.append("".join(sign_rec))
        
    return " ".join(sign_rec_words).strip()

In [9]:
# Updates the dataframe (change the old sign_info column to one containing the return of the new function)

def update_sign_info(row):
    if row['scroll'] != 'MT':
        return sign_info(TFOb(int(row['verb_id']), DSS))
    return row['sign_info']

# Apply the function to the dataframe
df['sign_info'] = df.apply(update_sign_info, axis=1)

In [10]:
# Initialize an empty list to collect mismatch information
mismatches = []

# Function to check the lengths of sign_info and gcons_verse
def check_length(row):
    if row['scroll'] != 'MT':
        if len(row['sign_info']) != len(row['gcons_verse']):
            mismatches.append(f"Mismatch found in row {row.name}: sign_info='{row['sign_info']}' (len={len(row['sign_info'])}), gcons_verse='{row['gcons_verse']}' (len={len(row['gcons_verse'])})")

# Apply the function to the dataframe
df.apply(check_length, axis=1)

# Check if there are any mismatches and print them
if mismatches:
    print("Mismatches found:")
    for mismatch in mismatches:
        print(mismatch)
else:
    print("No mismatches found.")

No mismatches found.


In [11]:
# The reconstruct_verse function adds a gcons version of the verse with brackets indicating the reconstructed sections

# Add a column with the verse plus brackets indicating reconstructed letters

def reconstruct_verse(sign_info, verse):
    rec_verse = []
    is_reconstructed = False

    for c_verse, c_sign in zip(verse, sign_info):
        if c_sign == "1" and not is_reconstructed:
            is_reconstructed = True
            rec_verse.append("[")
        if c_sign == "0" and is_reconstructed:
            is_reconstructed = False
            if rec_verse[-1] == " ":
                rec_verse.pop()
                rec_verse.extend(["]", " "])
            else:
                rec_verse.append("]")
        rec_verse.append(c_verse)

    if is_reconstructed:
        rec_verse.append("]")
    return "".join(rec_verse)


df["reconstructed_verse"] = ""

df["reconstructed_verse"] = df.apply(lambda row: reconstruct_verse(row.sign_info, row.gcons_verse) if row.book != "MT" else "", axis=1)


In [12]:
# Delete the DSS duplicated

verb_ids = df.verb_id.to_list()

duplicates = set([verb_id for verb_id in verb_ids if verb_ids.count(verb_id) > 1])


df = df[~((df.gcons_clause == "None") & (df.subject == "None") & (df.verb_id.isin(duplicates)))] # delete duplicates
df.reset_index(drop=True, inplace=True) # reset the index for AnnotEasy

In [13]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments', 'reconstructed_verse'],
      dtype='object')

In [14]:
# Add the spatial_arg_type column

df["spatial_arg_type"] = ""

In [15]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments', 'reconstructed_verse',
       'spatial_arg_type'],
      dtype='object')

In [16]:
# Remove the columns that have MN as preposition 1 and not other prepositions

In [17]:
# have a look at the data
df_test = df[(df.preposition_1 == "MN") & (df.preposition_2 == "")]
df_test.head(5)
#len(df_test)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type
2,310860,BRX[,MT,Psalms,3,1,BRXW,MZMWR L DWD B BRXW M PNJ >BCLWM BNW,B BRXW M PNJ >BCLWM BNW,,M PNJ >BCLWM BNW,MN PNH/ >BCLWM/ BN/,0.0,prep,pers,anim,det,subs,complex,,,qal,infc,MN,,,,,,,,,,,
9,311250,SWR[,MT,Psalms,6,9,SWRW,SWRW MMNJ KL P<LJ >WN KJ CM< JHWH QWL BKJJ,SWRW MMNJ,,MMNJ,MN,0.0,prep,,,det,,simple,,,qal,impv,MN,,,,,,,,,,,
66,312799,<BR[,MT,Psalms,18,13,<BRW,M NGH NGDW <BJW <BRW BRD W GXLJ >C,M NGH NGDW <BJW <BRW,<BJW,M NGH NGDW,MN NGH/ NGD/,0.0,prep,,,det,subs,complex,,,qal,perf,MN,,,,,,,,,,,
77,312906,SWR[,MT,Psalms,18,23,>SJR,KJ KL MCPVJW L NGDJ W XQTJW L> >SJR MNJ,W XQTJW L> >SJR MNJ,,MNJ,MN,0.0,prep,,,det,,simple,,,hif,impf,MN,,,,,,,,,,,
95,313234,JY>[,MT,Psalms,19,6,JY>,W HW> K XTN JY> M XPTW JFJF K GBWR L RWY >RX,JY> M XPTW,,M XPTW,MN XPH/,0.0,prep,,,det,subs,simple,,,qal,ptca,MN,,,,,,,,,,,


In [18]:
# Identify the rows to drop based on the condition
index_to_drop = df[(df.preposition_1 == "MN") & (df.preposition_2 == "")].index

# Drop the rows by index
df = df.drop(index_to_drop)

# Reset the index
df = df.reset_index(drop=True)

In [19]:
# Verification
df[(df.preposition_1 == "MN") & (df.preposition_2 == "")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type


In [20]:
df.head(20)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type
0,310658,HLK[,MT,Psalms,1,1,HLK,>CRJ H >JC >CR L> HLK B <YT RC<JM W B DRK XV>J...,>CR L> HLK B <YT RC<JM,,B <YT RC<JM,B <YH/ RC</,0.0,prep,,,det,subs,complex,,,qal,perf,B,,,,,,,,,,,
1,310854,XSH[,MT,Psalms,2,12,XWSJ,NCQW BR PN J>NP W T>BDW DRK KJ JB<R K M<V >PW ...,XWSJ BW,,BW,B,0.0,prep,,,det,,simple,,,qal,ptca,B,,,,,,,,,,,
2,311104,BW>[,MT,Psalms,5,8,>BW>,W >NJ B RB XSDK >BW> BJTK >CTXWH >L HJKL QDCK ...,W >NJ B RB XSDK >BW> BJTK,>NJ,BJTK,BJT/,0.0,vc,,,det,subs,simple,,,qal,impf,,,,,,,,,,,,
3,311136,NPL[,MT,Psalms,5,11,JPLW,H>CJMM >LHJM JPLW M M<YWTJHM B RB PC<JHM HDJXM...,JPLW M M<YWTJHM,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,,,
4,311149,XSH[,MT,Psalms,5,12,XWSJ,W JFMXW KL XWSJ BK L <WLM JRNNW W TSK <LJMW W ...,XWSJ BK,,BK,B,0.0,prep,,,det,,simple,,,qal,ptca,B,,,,,,,,,,,
5,2054155,XSH[,4Q98b,Psalms,5,12,XSJ,W JFMXW KL XSJ BKH L <WLM JRNNW W TSKH <LJMW W...,XSJ BKH,,BKH,,0.0,,,,,,,0.0,1 11111 00 000 000 0 1111 11111 1 1111 11111 1...,qal,ptca,BKH,,,,,,,,,,[W JFMXW] KL XSJ BKH L [<WLM JRNNW W TSKH <LJM...,
6,2046935,XSH[,4Q83,Psalms,5,12,XSJ,W JFMXW KWL XSJ BKH L <WLM JRNNW W TSK <LJMW W...,XSJ BKH,,BKH,,0.0,,,,,,,0.0,1 11111 111 000 000 0 0000 00011 1 111 11111 1...,qal,ptca,BKH,,,,,,,,,,[W JFMXW KWL] XSJ BKH L <WLM JRN[NW W TSK <LJM...,
7,311212,CWB[,MT,Psalms,6,5,CWBH,CWBH JHWH XLYH NPCJ HWCJ<NJ LM<N XSDK,CWBH,,no complement,no complement,,,,,,,,,,qal,impv,,,,,,,,,,,,
8,311272,CWB[,MT,Psalms,6,11,JCBW,JBCW W JBHLW M>D KL >JBJ JCBW JBCW RG<,JCBW,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,,,
9,311290,XSH[,MT,Psalms,7,2,XSJTJ,JHWH >LHJ BK XSJTJ HWCJ<NJ M KL RDPJ W HYJLNJ,BK XSJTJ,,BK,B,0.0,prep,,,det,,simple,,,qal,perf,B,,,,,,,,,,,


In [21]:
df.to_csv(PATH, index=False)